### 1. A* Search

In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import heapq
import random

class PuzzleState:
    def __init__(self, state, parent=None, action=None, cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.cost = cost
        self.size = len(state)
        
    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.state])
        
    def find_blank(self):
        return next((i, j) for i in range(self.size) 
                   for j in range(self.size) if self.state[i][j] == 0)

def generate_random_state(n):
    while True:
        numbers = list(range(n*n))
        random.shuffle(numbers)
        state = [numbers[i:i+n] for i in range(0, n*n, n)]
        if is_solvable(state):
            return state

def get_successors(node):
    blank_i, blank_j = node.find_blank()
    moves = [(0, 1, 'R'), (1, 0, 'D'), (0, -1, 'L'), (-1, 0, 'U')]
    successors = []
    
    for di, dj, action in moves:
        new_i, new_j = blank_i + di, blank_j + dj
        if 0 <= new_i < node.size and 0 <= new_j < node.size:
            new_state = [row[:] for row in node.state]
            new_state[blank_i][blank_j], new_state[new_i][new_j] = \
                new_state[new_i][new_j], new_state[blank_i][blank_j]
            successors.append(PuzzleState(new_state, node, action, node.cost + 1))
    return successors

def get_inversions(state):
    flat = []
    for i in range(len(state)):
        for j in range(len(state)):
            if state[i][j] != 0:
                flat.append(state[i][j])
    
    inversions = 0
    for i in range(len(flat)):
        for j in range(i + 1, len(flat)):
            if flat[i] > flat[j]:
                inversions += 1
    return inversions

def is_solvable(state):
    n = len(state)
    inversions = get_inversions(state)
    
    blank_row = 0
    for i in range(n-1, -1, -1):
        for j in range(n):
            if state[i][j] == 0:
                blank_row = n - i
                break
    
    if n % 2 == 1:
        return inversions % 2 == 0
    else:
        if blank_row % 2 == 0:
            return inversions % 2 == 1
        else:
            return inversions % 2 == 0

def misplaced_tiles(state, goal):
    count = 0
    for i in range(len(state)):
        for j in range(len(state)):
            if state[i][j] != 0 and state[i][j] != goal[i][j]:
                count += 1
    return count

def manhattan_distance(state, goal):
    n = len(state)
    distance = 0
    goal_pos = {}
    for i in range(n):
        for j in range(n):
            goal_pos[goal[i][j]] = (i, j)
    
    for i in range(n):
        for j in range(n):
            if state[i][j] != 0:
                gi, gj = goal_pos[state[i][j]]
                distance += abs(i - gi) + abs(j - gj)
    return distance

def astar_solve(initial_state, goal_state, heuristic_func, timeout=60):
    if not is_solvable(initial_state):
        return None, 0, 0
        
    start_time = time.time()
    initial = PuzzleState(initial_state)
    frontier = [(0 + heuristic_func(initial_state, goal_state), 0, id(initial), initial)]
    heapq.heapify(frontier)
    visited = set()
    nodes_expanded = 0
    
    while frontier and (time.time() - start_time) < timeout:
        _, cost, _, current = heapq.heappop(frontier)
        state_tuple = tuple(map(tuple, current.state))
        
        if current.state == goal_state:
            path = []
            while current.parent:
                path.append(current.action)
                current = current.parent
            return path[::-1], time.time() - start_time, nodes_expanded
            
        if state_tuple not in visited:
            visited.add(state_tuple)
            nodes_expanded += 1
            for successor in get_successors(current):
                if tuple(map(tuple, successor.state)) not in visited:
                    h = heuristic_func(successor.state, goal_state)
                    f = successor.cost + h
                    heapq.heappush(frontier, (f, successor.cost, id(successor), successor))
    
    return None, time.time() - start_time, nodes_expanded

def run_experiment(n, algorithm, heuristic_func, heuristic_name):
    goal_state = [[i + 1 if i < n*n-1 else 0 for i in range(j*n, (j+1)*n)] 
                 for j in range(n)]
    
    results = []
    nodes_expanded_list = []
    timeouts = 0
    initial_states = [generate_random_state(n) for _ in range(10)]
    
    for i, init_state in enumerate(initial_states, 1):
        print(f"\nTrial {i}/10 for {n}x{n} puzzle using {heuristic_name}")
        print("Initial State:")
        print(PuzzleState(init_state))
        print("\nGoal State:")
        print(PuzzleState(goal_state))
        
        solution, solve_time, nodes = algorithm(init_state, goal_state, heuristic_func)
        
        if solution is not None and solve_time < 60:
            results.append(solve_time)
            nodes_expanded_list.append(nodes)
            print(f"Solved in {solve_time:.2f} seconds")
            print(f"Nodes expanded: {nodes}")
            print("Moves:", ' '.join(solution))
        else:
            timeouts += 1
            print("Timeout or no solution")
    
    if results:
        avg_time = sum(results) / len(results)
        avg_nodes = sum(nodes_expanded_list) / len(nodes_expanded_list)
        std_dev = np.std(results) if len(results) > 1 else 0
        print(f"\nResults for n={n}:")
        print(f"Average time: {avg_time:.2f} seconds")
        print(f"Average nodes expanded: {avg_nodes:.2f}")
        print(f"Standard deviation: {std_dev:.2f}")
        print(f"Successful solves: {len(results)}/10")
        return avg_time, avg_nodes, std_dev, len(results)
    return None, None, None, 0

def main():
    n_values = [2, 3]
    heuristics = {
        'Misplaced Tiles': misplaced_tiles,
        'Manhattan Distance': manhattan_distance,
    }
    
    astar_results = {name: [] for name in heuristics}
    
    for n in n_values:
        print(f"\nTesting for n={n}")
        for heur_name, heur_func in heuristics.items():
            print(f"\nRunning A* with {heur_name}")
            avg_time, avg_nodes, _, _ = run_experiment(
                n, astar_solve, heur_func, f"A* - {heur_name}")
            if avg_time is not None:
                astar_results[heur_name].append((avg_time, avg_nodes))

if __name__ == "__main__":
    main()


Testing for n=2

Running A* with Misplaced Tiles

Trial 1/10 for 2x2 puzzle using A* - Misplaced Tiles
Initial State:
1 2
3 0

Goal State:
1 2
3 0
Solved in 0.00 seconds
Nodes expanded: 0
Moves: 

Trial 2/10 for 2x2 puzzle using A* - Misplaced Tiles
Initial State:
1 0
3 2

Goal State:
1 2
3 0
Solved in 0.00 seconds
Nodes expanded: 1
Moves: D

Trial 3/10 for 2x2 puzzle using A* - Misplaced Tiles
Initial State:
2 0
1 3

Goal State:
1 2
3 0
Solved in 0.00 seconds
Nodes expanded: 3
Moves: L D R

Trial 4/10 for 2x2 puzzle using A* - Misplaced Tiles
Initial State:
0 2
1 3

Goal State:
1 2
3 0
Solved in 0.00 seconds
Nodes expanded: 2
Moves: D R

Trial 5/10 for 2x2 puzzle using A* - Misplaced Tiles
Initial State:
0 1
3 2

Goal State:
1 2
3 0
Solved in 0.00 seconds
Nodes expanded: 2
Moves: R D

Trial 6/10 for 2x2 puzzle using A* - Misplaced Tiles
Initial State:
1 0
3 2

Goal State:
1 2
3 0
Solved in 0.00 seconds
Nodes expanded: 1
Moves: D

Trial 7/10 for 2x2 puzzle using A* - Misplaced Tiles
I